# Spin-Spin Hamiltonian Gate Fitting, with Decoherence

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from timedependent_channels import *
from utils import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=5, suppress=True)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


# Spin-Spin Hamiltonian, Noise Fitting

## Target Channel

In [2]:
tf.random.set_seed(42)
np.random.seed(42)

jump_operator = JumpOperator(4, trainable=False)
H_target = SpinSpin(degree=2)

lindblad_target = LindbladGenerator(hamiltonian = H_target, 
                                   jump_operator = jump_operator,
                                   gamma = 0,
                                 )

channel_target = MagnusPropagator(liouvillian=lindblad_target, grid_size=200, T = 1)

## Decoherence Optimization

In [3]:
tf.random.set_seed(43)
np.random.seed(43)

jump_operator = JumpOperator(4, trainable=True)
H_model = SpinSpin(degree=4)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 1,
                                 )
channel_model1 = MagnusPropagator(liouvillian=lindblad_model, grid_size=100, T = 1)

In [4]:
model1 = ModelQuantumMap(channel = channel_model1,
                        loss_function = channel_mse_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss, sample_freq=1, N=0),
                        )

In [5]:
model1.train(inputs=None,
             targets=[channel_target],
             num_iter=500,
             N=0,
             verbose=False,)

-0.7831295172654135 None


## No Decoherence Optimization

In [7]:
tf.random.set_seed(44)
np.random.seed(44)

jump_operator = deepcopy(jump_operator)
jump_operator.set_trainable(False)
H_model = SpinSpin(degree=4)

lindblad_model2 = LindbladGenerator(hamiltonian = H_model, 
                                    jump_operator = jump_operator,
                                    gamma = 0,
                                   )
channel_model2 = MagnusPropagator(liouvillian=lindblad_model2, grid_size=100, T = 1)

In [8]:
model2 = ModelQuantumMap(channel = channel_model2,
                        loss_function = channel_mse_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss, sample_freq=1, N=0),
                        )

In [9]:
model2.train(inputs=None,
            targets=[channel_target],
            num_iter=500,
            N=0,
            verbose=False,)
lindblad_model2.gamma = 1

-1.0000000746971471 None


In [10]:
print(channel_fidelity(channel_target, channel_model1))
print(channel_fidelity(channel_target, channel_model2))

tf.Tensor(0.783129528271472, shape=(), dtype=float64)
tf.Tensor(0.7839743159815153, shape=(), dtype=float64)


## Noise Fitting with Jump Operators with Non-Zero Trace

In [11]:
tf.random.set_seed(43)
np.random.seed(43)

jump_operator = JumpOperator(4, trainable=True, trace_less = False)
H_model = SpinSpin(degree=4)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 1,
                                 )
channel_model3 = MagnusPropagator(liouvillian=lindblad_model, grid_size=100, T = 1)

In [12]:
model3 = ModelQuantumMap(channel = channel_model3,
                        loss_function = channel_mse_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss, sample_freq=1, N=0),
                        )

In [13]:
model3.train(inputs=None,
            targets=[channel_target],
            num_iter=500,
            N=0,
            verbose=False,)

-0.9894551142049954 None


In [14]:
print(channel_fidelity(channel_target, channel_model3))

tf.Tensor(0.9894551285749733, shape=(), dtype=float64)


In [15]:
print(tf.linalg.trace(channel_model3.liouvillian.JumpOperator([0])[0]))
print(tf.linalg.trace(channel_model2.liouvillian.JumpOperator([0])[0]))

tf.Tensor((-0.003940933016020049-1.957060014560596j), shape=(), dtype=complex128)
tf.Tensor((-1.3877787807814457e-17-6.938893903907228e-18j), shape=(), dtype=complex128)


In [16]:
print(channel_model3.liouvillian.JumpOperator([0])[0])
print(channel_model2.liouvillian.JumpOperator([0])[0])

tf.Tensor(
[[ 0.0313 -0.33915j  0.00892-0.00904j  0.04614+0.00774j -0.01553+0.00297j]
 [ 0.00171-0.00217j -0.04658-0.55112j -0.00767+0.06738j -0.00802+0.00313j]
 [-0.00025+0.0037j  -0.0045 -0.00667j  0.00342-0.54623j  0.00684-0.00055j]
 [-0.00019+0.0007j  -0.02997-0.00005j -0.00376+0.0187j   0.00792-0.52057j]], shape=(4, 4), dtype=complex128)
tf.Tensor(
[[ 0.30388+0.2691j  -0.01065+0.01807j  0.26736+0.13488j -0.34904+0.14117j]
 [ 0.07883-0.04059j -0.25648-0.27485j -0.10572+0.17484j -0.05067-0.10565j]
 [ 0.16819+0.2884j   0.00838+0.09133j -0.104  -0.01651j  0.26999+0.17855j]
 [ 0.00863+0.1343j  -0.2544 -0.06622j -0.14931+0.22308j  0.0566 +0.02226j]], shape=(4, 4), dtype=complex128)
